In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from numpy import mean, argmax

import train
import loadDataset
import model1

### Load Dataset

In [2]:
X, Y = loadDataset.loadDataSet('./dataset/imgs/rock_frames', './dataset/imgs/paper_frames',
                               './dataset/imgs/scissor_frames', './dataset/csvs/rock.csv',
                               './dataset/csvs/paper.csv', './dataset/csvs/scissor.csv')
X = (X.astype(np.float32) - 128) / 128  # Normalize [-1,1]
Y = (Y > 0).astype(int)  # 0: None/NA, 1: Rock/Paper/Scissors

print("X:", X.shape, "Y:", Y.shape)

X: (10962, 64, 64, 3) Y: (10962,)


### Custom Cross-Validation Function
(N-45) 45-frame videos are too many to load all at once.

In [3]:
video_len = 45

# Custom Batch Generator
def _batches_video(inputs, batch_size, shuffle=False, allow_smaller_final_batch=False):
    if not isinstance(inputs, list) and not isinstance(inputs, tuple):
        raise TypeError("Inputs must be of type list or tuple.")
    if not all([isinstance(x, np.ndarray) for x in inputs]):
        raise TypeError("Each input in the input list must be a numpy array.")
    total_size = inputs[0].shape[0]
    if not all([x.shape[0] == total_size for x in inputs]):
        raise RuntimeError("All inputs must have equal first dimension.")

    total_size -= video_len
    order = np.arange(total_size) if shuffle is False else np.random.permutation(total_size)
    for i in range(int(total_size / batch_size)):
        start_indices = order[(i)*batch_size:(i+1)*batch_size]                    
        yield [np.concatenate([x[None, (start):(start+video_len)] for start in start_indices]) for x in inputs]
    if allow_smaller_final_batch:
        start_indices = order[int(total_size / batch_size)*batch_size:]                    
        yield [np.concatenate([x[None, (start):(start+video_len)] for start in start_indices]) for x in inputs]

In [4]:
# Custom Cross-Validation that uses Custom Batch Generator
def cross_validation(session, model, x, y, batch_size=64, epochs=5, K=5, shuffle=False, verbose=True, print_interval=100):
    # https://stackoverflow.com/questions/39748660/how-to-perform-k-fold-cross-validation-with-tensorflow
    [predict_op, loss_op, accuracy_op, train_op], (X, Y, training) = model

    # K-Fold Loop
    train_loss, train_accuracy = [], []
    valid_loss, valid_accuracy = [], []
    k = 0
    for train_i, valid_i in KFold(n_splits=K).split(x):
        train_loss.append([])
        train_accuracy.append([])

        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())

        num_train_batches = int(x[train_i].shape[0] / batch_size)
        num_valid_batches = int(x[valid_i].shape[0] / batch_size)

        # Training
        for e in range(epochs):
            sum_loss, sum_accuracy = 0, 0
            for batch_i, (batch_x, batch_y) in enumerate(_batches_video([x[train_i], y[train_i]], batch_size=batch_size, shuffle=shuffle, allow_smaller_final_batch=False)):
                batch_y = batch_y[:, -1]
                loss, accuracy, _ = session.run([loss_op, accuracy_op, train_op],
                                                feed_dict={X: batch_x, Y: batch_y, training: True})

                if verbose and batch_i % print_interval == 0:
                    print("Train Batch {}: Loss = {}, Accuracy = {}".format(batch_i + 1, loss, accuracy))
                sum_loss += loss
                sum_accuracy += accuracy

            train_loss[k].append(sum_loss / num_train_batches)
            train_accuracy[k].append(sum_accuracy / num_train_batches)
            if verbose:
                print("Epoch {}: Average Train Loss = {}, Average Train Accuracy = {}\n"
                      .format(e + 1, train_loss[k][e], train_accuracy[k][e]))

        # Validation
        sum_loss, sum_accuracy = 0, 0
        for batch_i, (batch_x, batch_y) in enumerate(_batches_video([x[valid_i], y[valid_i]], batch_size=batch_size, shuffle=shuffle, allow_smaller_final_batch=False)):
            batch_y = batch_y[:, -1]
            loss, accuracy = session.run([loss_op, accuracy_op],
                                         feed_dict={X: batch_x, Y: batch_y, training: False})

            if verbose and batch_i % print_interval == 0:
                print("Valid Batch {}: Loss = {}, Accuracy = {}".format(batch_i + 1, loss, accuracy))
            sum_loss += loss
            sum_accuracy += accuracy

        valid_loss.append(sum_loss / num_valid_batches)
        valid_accuracy.append(sum_accuracy / num_valid_batches)
        if verbose:
            print("Fold {}: Validation Loss = {}, Validation Accuracy = {}\n"
                  .format(k + 1, valid_loss[k], valid_accuracy[k]))

        k += 1

    # Results
    print("Average Valid Loss = {}, Average Valid Accuracy = {}".format(mean(valid_loss), mean(valid_accuracy)))

    plt.figure()
    plt.title("Training Loss per Epoch")
    plt.plot(np.arange(epochs), np.array(train_loss).T)
    plt.xlabel("Epoch")
    plt.xticks(np.arange(epochs), np.arange(epochs))
    plt.ylabel("Loss")
    plt.legend(["Fold %d" % i for i in range(1, K+1)])
    plt.show()

In [ ]:
test_batches = _batches_video([X, Y], batch_size=64, shuffle=True, allow_smaller_final_batch=False)
for Xb, Yb in test_batches:
    Xt, Yt = tf.placeholder(tf.float32, (None, 45, 64, 64, 3)), tf.placeholder(tf.int64, None)
    new_shape = [-1]+[d for i, d in enumerate(Xt.shape[1:]) if i != 1-1]
    
    conc = []
    for i in range(Xt.shape[1]):
        Xt2 = tf.reshape(Xt[:, i], new_shape)
        l = tf.layers.conv2d(Xt2, filters=1, kernel_size=[7,7], strides=[4,4], padding="valid")
        conc.append(tf.expand_dims(l, 1))
    print(tf.concat(conc, axis=1))
    break

In [ ]:
Xt = tf.placeholder(tf.float32, (None, 45, 64, 64, 3))
x1 = tf.reshape(Xt[:, 0], (-1, 64, 64, 3))
x2 = tf.reshape(Xt[:, 1], (-1, 64, 64, 3))
print(Xt.shape)

In [ ]:
print(X.shape[1:])

### Start Tensorflow Session

In [5]:
session = tf.Session()

### Construct Model

In [6]:
model = model1.construct(X.shape[1:])

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



### Train

In [ ]:
cross_validation(session, model, X, Y, epochs=5, shuffle=True, print_interval=10)

Train Batch 1: Loss = 0.7310421466827393, Accuracy = 0.484375
Train Batch 11: Loss = 0.5983849763870239, Accuracy = 0.625
Train Batch 21: Loss = 0.42891061305999756, Accuracy = 0.859375
Train Batch 31: Loss = 0.3024279475212097, Accuracy = 0.890625
Train Batch 41: Loss = 0.18213023245334625, Accuracy = 0.890625
Train Batch 51: Loss = 0.20979221165180206, Accuracy = 0.890625
Train Batch 61: Loss = 0.11843259632587433, Accuracy = 0.9375
Train Batch 71: Loss = 0.18731680512428284, Accuracy = 0.9375
Train Batch 81: Loss = 0.15587764978408813, Accuracy = 0.9375
Train Batch 91: Loss = 0.22917750477790833, Accuracy = 0.921875
Train Batch 101: Loss = 0.09380890429019928, Accuracy = 0.984375
Train Batch 111: Loss = 0.22687065601348877, Accuracy = 0.875
Train Batch 121: Loss = 0.1690642535686493, Accuracy = 0.9375
Train Batch 131: Loss = 0.26629316806793213, Accuracy = 0.90625
Epoch 1: Average Train Loss = 0.2712198434955012, Average Train Accuracy = 0.8731751824817519

Train Batch 1: Loss = 0.1